## Q & A on private documents

## Required setup

In [1]:
! pip install python-dotenv

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [43]:
! pip install tiktoken

     ---------------------------------------- 0.0/635.3 kB ? eta -:--:--
     - ----------------------------------- 30.7/635.3 kB 435.7 kB/s eta 0:00:02
     ------- ------------------------------ 122.9/635.3 kB 1.2 MB/s eta 0:00:01
     ----------------------------------- -- 593.9/635.3 kB 4.1 MB/s eta 0:00:01
     -------------------------------------- 635.3/635.3 kB 4.4 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/de/cd/d80c9e284ae6c1b2172dacf0651d25b78ee1f7efbc12d74ea7b87c766263/regex-2023.8.8-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/268.3 kB ? eta -:--:--
   --------------------------------------- 268.3/268.3 kB 16.1 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [48]:
! pip install pinecone-client

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/98/17/3675b83dca0a032d2750bf04fbfdf78a6e46fa3056eefc2574cdd14661d9/pinecone_client-2.2.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/60.0 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.0 kB 1.4 MB/s eta 0:00:01
     ---------------------------------------- 60.0/60.0 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for dnspython>=2.0.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for tqdm>=4.64.1 from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 682.7 kB/s e

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [52]:
import os
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader, WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)
pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'),environment=os.environ.get('PINECONE_ENV'))

In [3]:
! pip install pypdf -q

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [4]:
! pip install langchain

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/fb/c3/01030edac782a84c3e8d76053bc31073f73f5454b18de14848aab5aca354/langchain-0.0.279-py3-none-any.whl.metadata
  Obtaining dependency information for PyYAML>=5.3 from https://files.pythonhosted.org/packages/b3/34/65bb4b2d7908044963ebf614fe0fdb080773fc7030d7e39c8d3eddcd4257/PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/10/81/230bec5713913a472003b71c8dd7e3572de44499e741df576fd6b39d82c8/SQLAlchemy-2.0.20-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/21/4e/452858698e53ddf06ea137eac268db535c9605394c27236f9986168dd82f/aiohttp-3.8.5-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.6.0,>=0.5.7 from https://files.pythonhosted.org/packages/97/5f/e7cc90f36152810cab

DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [30]:
def load_document(file):

    name, extension = os.path.splitext(file)

    if extension == '.pdf' :
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    else:
        print(f"Document format {extension} is not supported!")
        
    # print(data[1].page_content)
    data = loader.load()
    return data

def load_from_wikipedia(query, lang='en', load_max_docs=2):
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data
    


In [36]:
def chunk_data(data,chunk_size=256):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=0)
    chunks=text_splitter.split_documents(data)
    return chunks
    

In [45]:
def print_embeddings_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts] )
    print(f'Total Token: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens/1000 * 0.0004:.6f}')
    

In [51]:
def insert_or_fetch_embeddings(index_name):

    enbeddings=OpenAIEmbeddings()
    

    if index_name in pinecone.list_indexs():
        print(f'index {index_name} already exists. Loading embeddings ...',end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('ok')
    else:
        print(f'Creating index {index_name} and embeddings ...',end='')
        pinecone.create_index(index_name, dimension=1536,metric='cosine')
        vector_store=Pinecone.from_documents(chunks,embeddings,index_name)
        print('ok')

    return vector_store

In [56]:
def delete_pinecone_index(index_name='all'):

    if index_name=='all':
        indexes = pinecone.list_indexes()
        print("Deleting al indexes....")
        for index in indexes:
            pinecone.delete_index(index)
        print("ok")

    else:
        print(f'Deleting index {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('ok')
    

## Running the Modules

In [37]:
## Running code
data = load_document('./data/SwinUnet.pdf')
# data = load_document('./Llama2_QuamtizedRun.ipynb')
# print(data[1].page_content)
# print(data[10].metadata)

print(f'There are {len(data)} pages in the pdf document')


Loading ./data/SwinUnet.pdf
There are 14 pages in the pdf document


In [39]:
chunks = chunk_data(data)
print(len(chunks))
print("$$$$$")
print(chunks[2].page_content)


155
$$$$$
neural networks based on U-shaped architecture and skip-connections
have been widely applied in a variety of medical image tasks. How-
ever, although CNN has achieved excellent performance, it cannot learn


In [46]:
print_embeddings_cost(chunks)

Total Token: 9383
Embedding Cost in USD: 0.003753


In [57]:
delete_pinecone_index()

TypeError: expected string or bytes-like object, got 'NoneType'

In [32]:
! pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     ----------- --------------------------- 41.0/143.0 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 143.0/143.0 kB 2.8 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11707 sha256=62afbc9308fed713c46f080ca56464b36d494b39869990906249abab1e16f5ff
  Stored in directory: c:\users\nisha\appdata\local\pip\cache\wheels\8f\ab\cb\45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


DEPRECATION: Loading egg at c:\users\nisha\miniconda3\envs\llama2conda\lib\site-packages\llama-0.0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [34]:
data = load_from_wikipedia('GPT-4')
print(data[0].page_content)

Generative Pre-trained Transformer 4 (GPT-4) is a multimodal large language model created by OpenAI, and the fourth in its series of GPT foundation models. It was initially released on March 14, 2023, and has been made publicly available via the paid chatbot product ChatGPT Plus, and via OpenAI's API.  As a transformer-based model, GPT-4 uses a paradigm where pre-training using both public data and "data licensed from third-party providers" is used to predict the next token. After this step, the model was then fine-tuned with reinforcement learning feedback from humans and AI for human alignment and policy compliance.: 2 Observers reported that the iteration of ChatGPT using GPT-4 was an improvement on the previous iteration based on GPT-3.5, with the caveat that GPT-4 retains some of the problems with earlier revisions. GPT-4 is also capable of taking images as input, though this feature has not been made available since launch. OpenAI has declined to reveal various technical details 